In [1]:
import nltk

nltk.download([
    "names",
    "stopwords",
    "averaged_perceptron_tagger",
    "vader_lexicon",
    "punkt",
    ])

[nltk_data] Downloading package names to /Users/eltontay/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eltontay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/eltontay/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/eltontay/nltk_data...
[nltk_data] Downloading package punkt to /Users/eltontay/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [1]:
import pandas as pd
import numpy as np

from newspaper import Article
from newspaper import Config

import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer

import string
import re

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/eltontay/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/eltontay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/eltontay/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
# df_world= pd.read_csv('../Data/Global/world_news.csv')
# df_politics= pd.read_csv('../Data/Global/politics_news.csv.csv')
# df_coronavirus= pd.read_csv('../Data/Global/coronavirus_news.csv')
df_aapl= pd.read_csv('../Data/News/Stock/aapl_news.csv')
df_meta= pd.read_csv('../Data/News/Stock/meta_news.csv')
df_tsla= pd.read_csv('../Data/News/Stock/tsla_news.csv')
# print('world shape ', df_world.shape)
# print('politics shape ',df_politics.shape)
# print('coronavirus shape ',df_coronavirus.shape)
print('aapl shape ',df_aapl.shape)
print('meta shape ', df_meta.shape)
print('tsla shape ', df_tsla.shape)
# print(df_world.head())
print(df_aapl.head())

aapl shape  (2289, 3)
meta shape  (2465, 3)
tsla shape  (1776, 3)
           Date                                              Title  \
0  Aug 31, 2022  Top 5 Things to Watch in Markets in the Week A...   
1  Aug 31, 2022  Dow Futures Move Higher As Investors Brace for...   
2  Aug 30, 2022  China's 20th Communist Party Congress: What yo...   
3  Aug 30, 2022  Top 5 Things to Watch in Markets in the Week A...   
4  Aug 30, 2022  Dow Futures Move Higher As Investors Brace for...   

                                                Text  
0  By Yasin Ebrahim\nInvesting.com -- The Dow stu...  
1  By Yasin Ebrahim\nInvesting.com -- The S&P 500...  
2  By Sam Boughedda\nAhead of Broadcom's (NASDAQ:...  
3  By Yasin Ebrahim\nInvesting.com -- The Dow clo...  
4  By Liz Moyer\nInvesting.com -- Stocks fell for...  


In [11]:
def remove_irrelevant_content(text):
    headline_only_string = "This headline-only article is meant to show you why a stock is moving, the most difficult aspect of stock trading"

    if headline_only_string in text:
        return ""
    else:
        return text

def remove_punctuation(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

def lowercase(text):
    return text.lower()

def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words('english')
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

def remove_special_character(text):
    text = text.replace('\n', ' ') 
    return text

def lemmatize(text):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in text]
    return corpus

def preprocess_text(text):
    text = remove_irrelevant_content(text)
    text = remove_punctuation(text)
    text = lowercase(text)
    # text = remove_stopwords(text)
    text = remove_special_character(text)
    # text = lemmatize(text)
    return text

In [15]:
sia = SentimentIntensityAnalyzer()

def vader_process(df) : 
    df['Processed Title'] = df['Title'].apply(lambda x: preprocess_text(x))
    df['Processed Text'] = df['Text'].apply(lambda x: preprocess_text(x))
    df['Sentiment Title'] = df['Processed Title'].apply(lambda x: sia.polarity_scores(x))
    df['Sentiment Text'] = df['Processed Text'].apply(lambda x: sia.polarity_scores(x))
    df['Positive Title'] = df['Sentiment Title'].apply(lambda x: x["compound"] > 0)
    df['Positive Text'] = df['Sentiment Text'].apply(lambda x: x["compound"] > 0)
    return df

In [16]:
df_aapl = vader_process(df_aapl)
df_aapl.head()
df_aapl.to_csv('../Data-Processed/News/Stock/aapl_vader.csv')

In [17]:
df_meta = vader_process(df_meta)
df_meta.head()
df_meta.to_csv('../Data-Processed/News/Stock/meta_vader.csv')

In [18]:
df_tsla = vader_process(df_tsla)
df_tsla.head()
df_tsla.to_csv('../Data-Processed/News/Stock/tsla_vader.csv')